In [5]:
from google.colab import drive
drive.mount('/content/drive')

import sys
sys.path.append('/content/drive/My Drive/Colab Notebooks/NeuralNetwork')

import numpy as np
import cupy as cp
from keras.datasets import mnist
import keras
from NeuralNetworkGPU import NeuralNetworkGPU

(x_train_np, t_train_np), (x_test_np, t_test_np) = mnist.load_data()

x_train_np  = x_train_np.reshape(60000, 784)
x_test_np   = x_test_np.reshape(10000, 784)
x_train_np  = x_train_np.astype('float32')
x_test_np   = x_test_np.astype('float32')
x_train_np /= 255
x_test_np  /= 255
t_train_np  = keras.utils.np_utils.to_categorical(t_train_np, 10)
t_test_np   = keras.utils.np_utils.to_categorical(t_test_np, 10)

x_train = cp.asarray(x_train_np)
t_train = cp.asarray(t_train_np)
x_test  = cp.asarray(x_test_np)
t_test  = cp.asarray(t_test_np) 

batch_size = 100

net = NeuralNetworkGPU()

net.add_affine(input = x_train.shape[1], output = 20, optimizer="SDG", lr = 0.01)
#net.add_batch_norm(size = 20, optimizer = "SDG", lr = 0.01)
net.add_activation_function(func = "ReLU", size = 20)

net.add_affine(input = 20, output = 20, optimizer="SDG", lr = 0.01)
#net.add_batch_norm(size = 20, optimizer = "SDG", lr = 0.01)
net.add_activation_function(func = "ReLU", size = 20)

net.add_affine(input = 20, output = 20, optimizer="SDG", lr = 0.01)
#net.add_batch_norm(size = 20, optimizer = "SDG", lr = 0.01)
net.add_activation_function(func = "ReLU", size = 20)

net.add_affine(input = 20, output = t_train.shape[1], optimizer="SDG", lr = 0.01)
#net.add_batch_norm(size =t_train.shape[1], optimizer = "SDG", lr = 0.01)
net.add_output_layer(func = "Softmax2CrossEntropy", size = t_train.shape[1])

net.create()
print("0エポック - " + "認識精度: " + "{:.2f}".format(net.accuracy(x_test, t_test)*100) + "%")  

for epoch in range(5):
    for i in range(int(x_train.shape[0]/batch_size)):
        batch_mask = np.random.choice(x_train.shape[0], batch_size)
        x_batch = x_train[batch_mask]
        t_batch = t_train[batch_mask]
        
        net.forward(x_batch, t_batch)
        net.backward()

    print("{}エポック - ".format(epoch + 1) + "認識精度: " + "{:.2f}".format(net.accuracy(x_test, t_test)*100) + "%")    


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
0エポック - 認識精度: 9.82%
1エポック - 認識精度: 10.40%
2エポック - 認識精度: 16.73%
3エポック - 認識精度: 18.65%
4エポック - 認識精度: 19.90%
5エポック - 認識精度: 21.24%


In [ ]:
#for i in range(5, 0, -1):
    print(i)

5
4
3
2
1
